    Arranging the data before applying the models

In [3]:
def Load_data():
    df = pd.read_csv('Whole_dataset1.txt')
    return df

### Sorting the data time wise 

In [4]:
def sort_by_time(df):
    df = df.sort_values(by='Date')
    return df

### Train test split

In [5]:
def tr_ts_split(df):
    l = int(df.shape[0]*0.7)
    ts_df = df[l:]; df = df[:l]
    return (df,ts_df)
    del df

### Joining tables

In [6]:
def join_all(tr_df, ts_df):
    movie_df = pd.concat([pd.read_pickle('info_1.pkl'),\
                     pd.read_pickle('info_2.pkl'),\
                     pd.read_pickle('info_3.pkl')])

    tr_df = tr_df.join(movie_df.set_index('MovieID'), on='MovieID')
    ts_df = ts_df.join(movie_df.set_index('MovieID'), on='MovieID')
    del movie_df

    cust_df = pd.concat([pd.read_pickle('movie_info1.pkl'),\
                     pd.read_pickle('movie_info2.pkl'),\
                     pd.read_pickle('movie_info3.pkl'),\
                    pd.read_pickle('movie_info4.pkl'),\
                    pd.read_pickle('movie_info5.pkl')])

    tr_df = tr_df.join(cust_df.set_index('CustomerID'), on='CustomerID')
    ts_df = ts_df.join(cust_df.set_index('CustomerID'), on='CustomerID')
    del cust_df
    
    about_movdf = pd.read_pickle('modified_movie_title_year.pkl')
    tr_df = tr_df.join(about_movdf.set_index('MovieID'), on='MovieID')
    ts_df = ts_df.join(about_movdf.set_index('MovieID'), on='MovieID')
    del about_movdf
    
    return (tr_df, ts_df)

In [7]:
from sklearn.metrics import mean_squared_error,r2_score
r2score,mse,model = [],[],[]

# Baseline models

### Linear Regression

    Preparing the data for the model

In [8]:
import pandas as pd
def make_data():
    df = Load_data()
    df = sort_by_time(df)
    tr_df, ts_df = tr_ts_split(df)
    del df
    tr_df, ts_df = join_all(tr_df,ts_df)
    return (tr_df, ts_df)

tr_df, ts_df = make_data()

In [9]:
tr_df.shape, ts_df.shape

((16837634, 11), (7216130, 11))

In [10]:
todrop = set(tr_df.columns)-{'Score','Cust_mean','MeanRating'}

In [11]:
tr_df = tr_df.drop(axis=1, labels=todrop)
ts_df = ts_df.drop(axis=1, labels=todrop)

In [12]:
tr_df.head(3)

,Score,MeanRating,Cust_mean
9056171,5,3.971317,3.364807
14892677,3,3.439929,3.364807
19585852,4,3.952726,3.364807


    The most basic version with just 2 features

In [13]:
import gc; gc.collect()

200

In [14]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs=-1,normalize=True)
reg.fit(tr_df.drop(axis=1,labels='Score'),tr_df.Score)
del tr_df
hypothesis = reg.predict(ts_df.drop(axis=1,labels='Score'))

In [15]:
mean_squared_error(ts_df.Score,hypothesis)

0.86481094843432604

In [16]:
r2_score(ts_df.Score,hypothesis)

0.26601405092785635

In [17]:
import math

In [18]:
mse.append(math.sqrt(mean_squared_error(ts_df.Score,hypothesis)))
r2score.append(r2_score(ts_df.Score,hypothesis))
model.append('Basic logistic regression')

In [19]:
del ts_df, hypothesis, reg

    Added MovieID and CustomerID. Theoritically it shouldn't change any result

In [20]:
tr_df, ts_df = make_data()

tr_df.shape, ts_df.shape

((16837634, 11), (7216130, 11))

In [21]:
todrop = set(tr_df.columns)-{'Score','Cust_mean','MeanRating','CustomerID','MovieID'}

In [22]:
tr_df = tr_df.drop(axis=1, labels=todrop)
ts_df = ts_df.drop(axis=1, labels=todrop)

In [23]:
tr_df.head(3)

,CustomerID,Score,MovieID,MeanRating,Cust_mean
9056171,510180,5,1798,3.971317,3.364807
14892677,510180,3,2866,3.439929,3.364807
19585852,510180,4,3730,3.952726,3.364807


In [24]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs=-1,normalize=True)
reg.fit(tr_df.drop(axis=1,labels='Score'),tr_df.Score)
del tr_df
hypothesis = reg.predict(ts_df.drop(axis=1,labels='Score'))

In [25]:
mean_squared_error(ts_df.Score,hypothesis)

0.86480092941179598

In [26]:
r2_score(ts_df.Score,hypothesis)

0.26602255431437538

In [27]:
mse.append(math.sqrt(mean_squared_error(ts_df.Score,hypothesis)))
r2score.append(r2_score(ts_df.Score,hypothesis))
model.append('Linear Regression with MovieID and CustomerID')

In [28]:
del ts_df, hypothesis, reg

In [29]:
gc.collect()

334

    Adding features: NumberOfrating, freq_of_rating

In [30]:
tr_df, ts_df = make_data()
tr_df.shape, ts_df.shape

((16837634, 11), (7216130, 11))

In [31]:
tr_df.columns

Index(['CustomerID', 'Score', 'Date', 'MovieID', 'NumberOfRating',
       'MeanRating', 'StdDev', 'Cust_mean', 'cust_stddev', 'freq_of_rating',
       'Year_of_release'],
      dtype='object')

In [32]:
todrop = set(tr_df.columns)-{'Score','Cust_mean',\
        'MeanRating','CustomerID','MovieID','NumberOfRating','freq_of_rating'}

In [33]:
tr_df = tr_df.drop(axis=1, labels=todrop)
ts_df = ts_df.drop(axis=1, labels=todrop)

In [34]:
tr_df.head(3)

,CustomerID,Score,MovieID,NumberOfRating,MeanRating,Cust_mean,freq_of_rating
9056171,510180,5,1798,113377,3.971317,3.364807,233
14892677,510180,3,2866,22415,3.439929,3.364807,233
19585852,510180,4,3730,40191,3.952726,3.364807,233


In [35]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs=-1,normalize=True)
reg.fit(tr_df.drop(axis=1,labels='Score'),tr_df.Score)
del tr_df
hypothesis = reg.predict(ts_df.drop(axis=1,labels='Score'))

In [36]:
mean_squared_error(ts_df.Score,hypothesis)

0.86279584799181686

In [37]:
r2_score(ts_df.Score,hypothesis)

0.26772431536593788

In [38]:
mse.append(math.sqrt(mean_squared_error(ts_df.Score,hypothesis)))
r2score.append(r2_score(ts_df.Score,hypothesis))
model.append('Linear Regression: added Numberofrating and freq_of_rating')

In [39]:
del ts_df, hypothesis, reg

In [40]:
gc.collect()

332

    All features except time related variables

In [41]:
tr_df, ts_df = make_data()
tr_df.shape, ts_df.shape

((16837634, 11), (7216130, 11))

In [42]:
tr_df.columns

Index(['CustomerID', 'Score', 'Date', 'MovieID', 'NumberOfRating',
       'MeanRating', 'StdDev', 'Cust_mean', 'cust_stddev', 'freq_of_rating',
       'Year_of_release'],
      dtype='object')

In [43]:
todrop = ['Date','Year_of_release']

In [44]:
tr_df = tr_df.drop(axis=1, labels=todrop)
ts_df = ts_df.drop(axis=1, labels=todrop)

In [45]:
tr_df.head(3)

,CustomerID,Score,MovieID,NumberOfRating,MeanRating,StdDev,Cust_mean,cust_stddev,freq_of_rating
9056171,510180,5,1798,113377,3.971317,0.893782,3.364807,0.958341,233
14892677,510180,3,2866,22415,3.439929,1.099609,3.364807,0.958341,233
19585852,510180,4,3730,40191,3.952726,0.888977,3.364807,0.958341,233


In [46]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs=-1,normalize=True)
reg.fit(tr_df.drop(axis=1,labels='Score'),tr_df.Score)
del tr_df
hypothesis = reg.predict(ts_df.drop(axis=1,labels='Score'))

In [47]:
mean_squared_error(ts_df.Score,hypothesis)

0.86270745204593635

In [48]:
r2_score(ts_df.Score,hypothesis)

0.26779933914119036

In [49]:
mse.append(math.sqrt(mean_squared_error(ts_df.Score,hypothesis)))
r2score.append(r2_score(ts_df.Score,hypothesis))
model.append('Linear Regression: added everything except time dependant variables')

In [50]:
del ts_df, hypothesis, reg
gc.collect()

376

## Conclusion

In [55]:
print ('Final Results: ')
for i in range(len(mse)):
    print ('{}----{}'.format(i,mse[i]))
print('\n')
for i in range(len(model)):
    print ('{}----{}'.format(i,model[i]))

Final Results: 
0----0.9299521215817114
1----0.92994673471753
2----0.9288680465985558
3----0.9288204627622803


0----Basic logistic regression
1----Linear Regression with MovieID and CustomerID
2----Linear Regression: added Numberofrating and freq_of_rating
3----Linear Regression: added everything except time dependant variables
